## Prepare the workspace

In [7]:
# Check torch version and CUDA status if GPU is enabled.
import torch
print(torch.__version__)
print(torch.cuda.is_available()) # Should return True when GPU is enabled.

2.5.1+cu121
True


# Developing an AI application

Going forward, AI algorithms will be incorporated into more and more everyday applications. For example, you might want to include an image classifier in a smart phone app. To do this, you'd use a deep learning model trained on hundreds of thousands of images as part of the overall application architecture. A large part of software development in the future will be using these types of models as common parts of applications.

In this project, you'll train an image classifier to recognize different species of flowers. You can imagine using something like this in a phone app that tells you the name of the flower your camera is looking at. In practice you'd train this classifier, then export it for use in your application. We'll be using [this dataset](http://www.robots.ox.ac.uk/~vgg/data/flowers/102/index.html) of 102 flower categories, you can see a few examples below.

<img src='assets/Flowers.png' width=500px>

The project is broken down into multiple steps:

* Load and preprocess the image dataset
* Train the image classifier on your dataset
* Use the trained classifier to predict image content

We'll lead you through each part which you'll implement in Python.

When you've completed this project, you'll have an application that can be trained on any set of labeled images. Here your network will be learning about flowers and end up as a command line application. But, what you do with your new skills depends on your imagination and effort in building a dataset. For example, imagine an app where you take a picture of a car, it tells you what the make and model is, then looks up information about it. Go build your own dataset and make something new.

First up is importing the packages you'll need. It's good practice to keep all the imports at the beginning of your code. As you work through this notebook and find you need to import a package, make sure to add the import up here.

In [8]:
# Imports here
# imports for working with PyTorch and deep learning
import torch
from torch import nn, optim  # Neural network modules and optimizers
from torchvision import datasets, transforms, models  # Datasets, transforms, and pre-trained models

# Imports for handling images and visualizations
from PIL import Image  # Image processing
import matplotlib.pyplot as plt  # Visualization of data and model outputs

# Utility imports for data handling and processing
import os  # File and directory management
import numpy as np  # Numerical operations

# Imports for downloading and extracting datasets
import requests  # For downloading files from the web
import tarfile  # For extracting tar.gz files
import shutil  # For file and directory management


## Load the data

Here you'll use `torchvision` to load the data ([documentation](http://pytorch.org/docs/0.3.0/torchvision/index.html)). The data should be included alongside this notebook, otherwise you can [download it here](https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz).

If you do not find the `flowers/` dataset in the current directory, **/workspace/home/aipnd-project/**, you can download it using the following commands.

**Method 1**
```bash
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!unlink flowers
!mkdir flowers && tar -xzf flower_data.tar.gz -C flowers
```


**Method 2**
```bash
!cp -r /data/ .
```

## Data Description
The dataset is split into three parts, training, validation, and testing. For the training, you'll want to apply transformations such as random scaling, cropping, and flipping. This will help the network generalize leading to better performance. You'll also need to make sure the input data is resized to 224x224 pixels as required by the pre-trained networks.

The validation and testing sets are used to measure the model's performance on data it hasn't seen yet. For this you don't want any scaling or rotation transformations, but you'll need to resize then crop the images to the appropriate size.

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`, calculated from the ImageNet images.  These values will shift each color channel to be centered at 0 and range from -1 to 1.


In [9]:
data_dir = 'flowers'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [10]:
!wget 'https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz'
!tar -xzf flower_data.tar.gz
!mv flower_data flowers


--2025-01-25 07:05:39--  https://s3.amazonaws.com/content.udacity-data.com/nd089/flower_data.tar.gz
Resolving s3.amazonaws.com (s3.amazonaws.com)... 54.231.201.40, 52.217.123.216, 54.231.128.56, ...
Connecting to s3.amazonaws.com (s3.amazonaws.com)|54.231.201.40|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 344873452 (329M) [application/x-gzip]
Saving to: ‘flower_data.tar.gz’

flower_data.tar.gz  100%[===================>] 328.90M  7.70MB/s    in 43s     

2025-01-25 07:06:23 (7.65 MB/s) - ‘flower_data.tar.gz’ saved [344873452/344873452]

mv: cannot stat 'flower_data': No such file or directory


In [11]:
!ls


flower_data.tar.gz  sample_data  test  train  valid


In [12]:
!mkdir flowers
!mv train valid test flowers/

In [13]:
!ls flowers


test  train  valid


In [14]:
from torchvision import datasets, transforms
import torch

# TODO: Define your transforms for the training, validation, and testing sets
# Transformations for training data: includes augmentation for generalization
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),  # Randomly crop and resize to 224x224
    transforms.RandomRotation(30),  # Randomly rotate the image by up to 30 degrees
    transforms.RandomHorizontalFlip(),  # Randomly flip the image horizontally
    transforms.ToTensor(),  # Convert image to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize using ImageNet statistics
])

# Transformations for validation and testing data: no augmentation, only resizing and normalization
valid_test_transforms = transforms.Compose([
    transforms.Resize(256),  # Resize the shortest side to 256 pixels
    transforms.CenterCrop(224),  # Center-crop the image to 224x224
    transforms.ToTensor(),  # Convert image to a PyTorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize using ImageNet statistics
])

# TODO: Load the datasets with ImageFolder
# Load training, validation, and testing datasets using ImageFolder
data_dir = 'flowers'
image_datasets = {
    'train': datasets.ImageFolder(root=f"{data_dir}/train", transform=train_transforms),
    'valid': datasets.ImageFolder(root=f"{data_dir}/valid", transform=valid_test_transforms),
    'test': datasets.ImageFolder(root=f"{data_dir}/test", transform=valid_test_transforms),
}

# TODO: Using the image datasets and the transforms, define the DataLoaders
# Create DataLoaders for efficient batch processing
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=64, shuffle=False),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=64, shuffle=False),
}

# Print confirmation that the DataLoaders are ready
print("DataLoaders are set up successfully!")
print(f"Training data: {len(image_datasets['train'])} images.")
print(f"Validation data: {len(image_datasets['valid'])} images.")
print(f"Testing data: {len(image_datasets['test'])} images.")


DataLoaders are set up successfully!
Training data: 6552 images.
Validation data: 818 images.
Testing data: 819 images.


### Label mapping

You'll also need to load in a mapping from category label to category name. You can find this in the file `cat_to_name.json`. It's a JSON object which you can read in with the [`json` module](https://docs.python.org/2/library/json.html). This will give you a dictionary mapping the integer encoded categories to the actual names of the flowers.

In [15]:
import json

# Define the category-to-name mapping
cat_to_name = {
    "1": "pink primrose",
    "2": "hard-leaved pocket orchid",
    "3": "canterbury bells",
    "4": "sweet pea",
    "5": "english marigold",
    "6": "tiger lily",
    "7": "moon orchid",
    "8": "bird of paradise",
    "9": "monkshood",
    "10": "globe thistle"
}

# Save the dictionary to a JSON file
with open('cat_to_name.json', 'w') as f:
    json.dump(cat_to_name, f)

print("cat_to_name.json has been created!")


cat_to_name.json has been created!


In [16]:
import json

with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Building and training the classifier

Now that the data is ready, it's time to build and train the classifier. As usual, you should use one of the pretrained models from `torchvision.models` to get the image features. Build and train a new feed-forward classifier using those features.

We're going to leave this part up to you. Refer to [the rubric](https://review.udacity.com/#!/rubrics/1663/view) for guidance on successfully completing this section. Things you'll need to do:

* Load a [pre-trained network](http://pytorch.org/docs/master/torchvision/models.html) (If you need a starting point, the VGG networks work great and are straightforward to use)
* Define a new, untrained feed-forward network as a classifier, using ReLU activations and dropout
* Train the classifier layers using backpropagation using the pre-trained network to get the features
* Track the loss and accuracy on the validation set to determine the best hyperparameters

We've left a cell open for you below, but use as many as you need. Our advice is to break the problem up into smaller parts you can run separately. Check that each part is doing what you expect, then move on to the next. You'll likely find that as you work through each part, you'll need to go back and modify your previous code. This is totally normal!

When training make sure you're updating only the weights of the feed-forward network. You should be able to get the validation accuracy above 70% if you build everything right. Make sure to try different hyperparameters (learning rate, units in the classifier, epochs, etc) to find the best model. Save those hyperparameters to use as default values in the next part of the project.

One last important tip if you're using the workspace to run your code: To avoid having your workspace disconnect during the long-running tasks in this notebook, please read in the earlier page in this lesson called Intro to
GPU Workspaces about Keeping Your Session Active.

## Note for Workspace users:
If your network is over 1 GB when saved as a checkpoint, there might be issues with saving backups in your workspace. Typically this happens with wide dense layers after the convolutional layers. If your saved checkpoint is larger than 1 GB (you can open a terminal and check with `ls -lh`), you should reduce the size of your hidden layers and train again.

In [17]:
!ls


cat_to_name.json  flower_data.tar.gz  flowers  sample_data


In [18]:
!ls flowers


test  train  valid


In [19]:
from torchvision import datasets, transforms

# Define the data directory
data_dir = 'flowers'

# Define transformations for training data (data augmentation + normalization)
train_transforms = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomRotation(30),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Define transformations for validation and testing data (normalization only)
valid_test_transforms = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Load the datasets using ImageFolder
image_datasets = {
    'train': datasets.ImageFolder(root=f"{data_dir}/train", transform=train_transforms),
    'valid': datasets.ImageFolder(root=f"{data_dir}/valid", transform=valid_test_transforms),
    'test': datasets.ImageFolder(root=f"{data_dir}/test", transform=valid_test_transforms)
}

# Create DataLoaders for batch processing
dataloaders = {
    'train': torch.utils.data.DataLoader(image_datasets['train'], batch_size=64, shuffle=True),
    'valid': torch.utils.data.DataLoader(image_datasets['valid'], batch_size=64, shuffle=False),
    'test': torch.utils.data.DataLoader(image_datasets['test'], batch_size=64, shuffle=False)
}

print("DataLoaders created successfully!")
print(f"Number of training samples: {len(image_datasets['train'])}")
print(f"Number of validation samples: {len(image_datasets['valid'])}")
print(f"Number of testing samples: {len(image_datasets['test'])}")


DataLoaders created successfully!
Number of training samples: 6552
Number of validation samples: 818
Number of testing samples: 819


In [20]:
# TODO: Build and train your network

import torch
from torch import nn, optim
from torchvision import models
from torchvision.models import VGG16_Weights

# Step 1: Load a Pre-trained Model
# Using a pre-trained VGG16 model with updated 'weights' parameter
weights = VGG16_Weights.DEFAULT  # Default weights from the latest torchvision versions
model = models.vgg16(weights=weights)

# Step 2: Freeze Pre-trained Parameters
# Freezing the feature extraction layers to avoid updating their weights
for param in model.features.parameters():
    param.requires_grad = False

# Step 3: Define a New Classifier
# Replace the pre-trained classifier with a custom one
input_features = model.classifier[0].in_features  # Number of input features in the pre-trained model
hidden_units = 4096  # Number of units in the hidden layer

classifier = nn.Sequential(
    nn.Linear(input_features, hidden_units),  # Input to hidden layer
    nn.ReLU(),  # Activation function
    nn.Dropout(0.2),  # Dropout for regularization
    nn.Linear(hidden_units, 102),  # Hidden layer to 102 output classes
    nn.LogSoftmax(dim=1)  # Log-Softmax for output probabilities
)

# Replace the classifier in the model
model.classifier = classifier

# Step 4: Define the Loss Function and Optimizer
criterion = nn.NLLLoss()  # Negative Log Likelihood Loss for classification
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)  # Optimizer for the classifier

# Step 5: Set the Device (GPU if available, else CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)  # Move the model to the selected device

# Step 6: Training the Network
epochs = 5  # Number of epochs to train the model
steps = 0  # Counter for tracking training progress
train_losses, valid_losses = [], []  # Lists to track loss trends

print("Starting training...")
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0

    for inputs, labels in dataloaders['train']:
        steps += 1
        inputs, labels = inputs.to(device), labels.to(device)  # Move data to the selected device

        optimizer.zero_grad()  # Clear the gradients from the previous step
        log_ps = model(inputs)  # Forward pass
        loss = criterion(log_ps, labels)  # Compute the loss
        loss.backward()  # Backward pass to calculate gradients
        optimizer.step()  # Update weights

        running_loss += loss.item()

    # Validation Loop
    model.eval()  # Set model to evaluation mode
    valid_loss = 0
    accuracy = 0

    with torch.no_grad():  # Disable gradient calculations for validation
        for inputs, labels in dataloaders['valid']:
            inputs, labels = inputs.to(device), labels.to(device)  # Move data to the selected device
            log_ps = model(inputs)  # Forward pass
            valid_loss += criterion(log_ps, labels).item()  # Compute validation loss

            # Calculate accuracy
            ps = torch.exp(log_ps)  # Convert log probabilities to probabilities
            top_p, top_class = ps.topk(1, dim=1)  # Get the top prediction
            equals = top_class == labels.view(*top_class.shape)  # Check if prediction is correct
            accuracy += torch.mean(equals.type(torch.FloatTensor)).item()  # Calculate batch accuracy

    # Track loss and accuracy
    train_losses.append(running_loss / len(dataloaders['train']))
    valid_losses.append(valid_loss / len(dataloaders['valid']))

    print(f"Epoch {epoch+1}/{epochs}.. "
          f"Train loss: {running_loss / len(dataloaders['train']):.3f}.. "
          f"Validation loss: {valid_loss / len(dataloaders['valid']):.3f}.. "
          f"Validation accuracy: {accuracy / len(dataloaders['valid']):.3f}")

print("Training completed!")

Starting training...
Epoch 1/5.. Train loss: 4.145.. Validation loss: 1.062.. Validation accuracy: 0.702
Epoch 2/5.. Train loss: 1.465.. Validation loss: 0.697.. Validation accuracy: 0.809
Epoch 3/5.. Train loss: 1.217.. Validation loss: 0.615.. Validation accuracy: 0.836
Epoch 4/5.. Train loss: 1.125.. Validation loss: 0.560.. Validation accuracy: 0.857
Epoch 5/5.. Train loss: 1.055.. Validation loss: 0.571.. Validation accuracy: 0.851
Training completed!


## Testing your network

It's good practice to test your trained network on test data, images the network has never seen either in training or validation. This will give you a good estimate for the model's performance on completely new images. Run the test images through the network and measure the accuracy, the same way you did validation. You should be able to reach around 70% accuracy on the test set if the model has been trained well.

In [21]:
# TODO: Do validation on the test set
# Set the model to evaluation mode
model.eval()

# Initialize variables to track test loss and accuracy
test_loss = 0
test_accuracy = 0

# Use torch.no_grad() to turn off gradient calculations for validation
with torch.no_grad():
    for inputs, labels in dataloaders['test']:
        # Move data to the appropriate device (GPU or CPU)
        inputs, labels = inputs.to(device), labels.to(device)

        # Forward pass: compute the model's output
        log_ps = model(inputs)

        # Calculate the test loss
        loss = criterion(log_ps, labels)
        test_loss += loss.item()

        # Calculate accuracy
        ps = torch.exp(log_ps)  # Convert log probabilities to probabilities
        top_p, top_class = ps.topk(1, dim=1)  # Get the top prediction for each input
        equals = top_class == labels.view(*top_class.shape)  # Check if predictions match the labels
        test_accuracy += torch.mean(equals.type(torch.FloatTensor)).item()

# Calculate the average test loss and accuracy
test_loss = test_loss / len(dataloaders['test'])
test_accuracy = test_accuracy / len(dataloaders['test'])

# Print the results
print(f"Test Loss: {test_loss:.3f}")
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


Test Loss: 0.574
Test Accuracy: 83.82%


## Save the checkpoint

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: `image_datasets['train'].class_to_idx`. You can attach this to the model as an attribute which makes inference easier later on.

```model.class_to_idx = image_datasets['train'].class_to_idx```

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, `optimizer.state_dict`. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [22]:
# TODO: Save the checkpoint
# Attach the class-to-index mapping to the model
model.class_to_idx = image_datasets['train'].class_to_idx

# Define the checkpoint dictionary
checkpoint = {
    'architecture': 'vgg16',  # Specify the model architecture
    'classifier': model.classifier,  # Save the classifier
    'state_dict': model.state_dict(),  # Save the model's learned parameters
    'class_to_idx': model.class_to_idx,  # Save the class-to-index mapping
    'optimizer_state': optimizer.state_dict(),  # Save the optimizer state
    'epochs': epochs  # Save the number of training epochs
}

# Save the checkpoint to a file
torch.save(checkpoint, 'model_checkpoint.pth')
print("Model checkpoint saved successfully!")


Model checkpoint saved successfully!


## Loading the checkpoint

At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [23]:
# TODO: Write a function that loads a checkpoint and rebuilds the model
import torch
from torchvision import models

def load_checkpoint(filepath):
    """
    Load a model checkpoint and rebuild the model.

    Args:
        filepath (str): Path to the checkpoint file.

    Returns:
        model: The rebuilt model with loaded weights and configurations.
    """
    # Load the checkpoint
    checkpoint = torch.load(filepath)

    # Rebuild the pre-trained model
    model = models.vgg16(pretrained=True) if checkpoint['architecture'] == 'vgg16' else None

    if model is None:
        raise ValueError(f"Model architecture '{checkpoint['architecture']}' is not supported.")

    # Replace the classifier with the one saved in the checkpoint
    model.classifier = checkpoint['classifier']

    # Load the model's state_dict (weights and biases)
    model.load_state_dict(checkpoint['state_dict'])

    # Attach the class-to-index mapping
    model.class_to_idx = checkpoint['class_to_idx']

    print(f"Model '{checkpoint['architecture']}' loaded successfully!")
    return model


# Inference for classification

Now you'll write a function to use a trained network for inference. That is, you'll pass an image into the network and predict the class of the flower in the image. Write a function called `predict` that takes an image and a model, then returns the top $K$ most likely classes along with the probabilities. It should look like

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

First you'll need to handle processing the input image such that it can be used in your network.

## Image Preprocessing

You'll want to use `PIL` to load the image ([documentation](https://pillow.readthedocs.io/en/latest/reference/Image.html)). It's best to write a function that preprocesses the image so it can be used as input for the model. This function should process the images in the same manner used for training.

First, resize the images where the shortest side is 256 pixels, keeping the aspect ratio. This can be done with the [`thumbnail`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) or [`resize`](http://pillow.readthedocs.io/en/3.1.x/reference/Image.html#PIL.Image.Image.thumbnail) methods. Then you'll need to crop out the center 224x224 portion of the image.

Color channels of images are typically encoded as integers 0-255, but the model expected floats 0-1. You'll need to convert the values. It's easiest with a Numpy array, which you can get from a PIL image like so `np_image = np.array(pil_image)`.

As before, the network expects the images to be normalized in a specific way. For the means, it's `[0.485, 0.456, 0.406]` and for the standard deviations `[0.229, 0.224, 0.225]`. You'll want to subtract the means from each color channel, then divide by the standard deviation.

And finally, PyTorch expects the color channel to be the first dimension but it's the third dimension in the PIL image and Numpy array. You can reorder dimensions using [`ndarray.transpose`](https://docs.scipy.org/doc/numpy-1.13.0/reference/generated/numpy.ndarray.transpose.html). The color channel needs to be first and retain the order of the other two dimensions.

In [24]:
from PIL import Image
import numpy as np
import torchvision.transforms as transforms
def process_image(image):
    ''' Scales, crops, and normalizes a PIL image for a PyTorch model,
        returns an Numpy array
    '''

    # TODO: Process a PIL image for use in a PyTorch model
    # Open the image using PIL
    image = Image.open(image_path)

    # Define the transformation pipeline
    preprocess = transforms.Compose([
        transforms.Resize(256),  # Resize the shortest side to 256 pixels
        transforms.CenterCrop(224),  # Center crop the image to 224x224
        transforms.ToTensor(),  # Convert the image to a PyTorch tensor
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # Normalize using ImageNet stats
    ])

    # Apply the transformations
    tensor_image = preprocess(image)

    # Convert the tensor to a NumPy array
    np_image = tensor_image.numpy()

    return np_image

To check your work, the function below converts a PyTorch tensor and displays it in the notebook. If your `process_image` function works, running the output through this function should return the original image (except for the cropped out portions).

In [25]:
def imshow(image, ax=None, title=None):
    """Imshow for Tensor."""
    if ax is None:
        fig, ax = plt.subplots()

    # PyTorch tensors assume the color channel is the first dimension
    # but matplotlib assumes is the third dimension
    image = image.numpy().transpose((1, 2, 0))

    # Undo preprocessing
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean

    # Image needs to be clipped between 0 and 1 or it looks like noise when displayed
    image = np.clip(image, 0, 1)

    ax.imshow(image)

    return ax

## Class Prediction

Once you can get images in the correct format, it's time to write a function for making predictions with your model. A common practice is to predict the top 5 or so (usually called top-$K$) most probable classes. You'll want to calculate the class probabilities then find the $K$ largest values.

To get the top $K$ largest values in a tensor use [`x.topk(k)`](http://pytorch.org/docs/master/torch.html#torch.topk). This method returns both the highest `k` probabilities and the indices of those probabilities corresponding to the classes. You need to convert from these indices to the actual class labels using `class_to_idx` which hopefully you added to the model or from an `ImageFolder` you used to load the data ([see here](#Save-the-checkpoint)). Make sure to invert the dictionary so you get a mapping from index to class as well.

Again, this method should take a path to an image and a model checkpoint, then return the probabilities and classes.

```python
probs, classes = predict(image_path, model)
print(probs)
print(classes)
> [ 0.01558163  0.01541934  0.01452626  0.01443549  0.01407339]
> ['70', '3', '45', '62', '55']
```

In [26]:
import torch
from torch import nn
from torchvision import models
def predict(image_path, model, topk=5):
    ''' Predict the class (or classes) of an image using a trained deep learning model.
    '''

    # TODO: Implement the code to predict the class from an image file
    # Process the image
    model.eval()  # Ensure the model is in evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)  # Move model to the appropriate device

    # Process the image and convert to tensor
    processed_image = process_image(image_path)
    image_tensor = torch.tensor(processed_image).unsqueeze(0).to(device)  # Add batch dimension and move to device

    # Disable gradient calculations for inference
    with torch.no_grad():
        # Forward pass through the model
        log_ps = model(image_tensor)

        # Convert log probabilities to probabilities
        ps = torch.exp(log_ps)

        # Get the top k probabilities and indices
        top_probs, top_indices = ps.topk(topk, dim=1)

    # Convert probabilities and indices to lists
    top_probs = top_probs.cpu().numpy().flatten().tolist()
    top_indices = top_indices.cpu().numpy().flatten().tolist()

    # Map the indices to the corresponding class labels
    idx_to_class = {v: k for k, v in model.class_to_idx.items()}  # Reverse the class_to_idx mapping
    top_classes = [idx_to_class[i] for i in top_indices]

    return top_probs, top_classes

## Sanity Checking

Now that you can use a trained model for predictions, check to make sure it makes sense. Even if the testing accuracy is high, it's always good to check that there aren't obvious bugs. Use `matplotlib` to plot the probabilities for the top 5 classes as a bar graph, along with the input image. It should look like this:

<img src='assets/inference_example.png' width=300px>

You can convert from the class integer encoding to actual flower names with the `cat_to_name.json` file (should have been loaded earlier in the notebook). To show a PyTorch tensor as an image, use the `imshow` function defined above.

In [27]:
# TODO: Display an image along with the top 5 classes
import matplotlib.pyplot as plt
import numpy as np

def imshow(image, ax=None, title=None):
    """
    Display a processed image.

    Args:
        image (np.ndarray): The processed image (C x H x W).
        ax (matplotlib.axes._axes.Axes, optional): Axis to plot the image on. Defaults to None.
        title (str, optional): Title for the image. Defaults to None.

    Returns:
        ax: The matplotlib axis object with the image.
    """
    if ax is None:
        fig, ax = plt.subplots()

    # Convert image to H x W x C for displaying
    image = image.transpose((1, 2, 0))

    # Reverse the normalization process
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    image = std * image + mean
    image = np.clip(image, 0, 1)  # Ensure pixel values are in [0, 1]

    ax.imshow(image)
    if title:
        ax.set_title(title)
    ax.axis('off')  # Remove axis ticks

    return ax

def display_image_with_predictions(image_path, model, topk=5):
    """
    Display an image along with the top predicted classes and their probabilities.

    Args:
        image_path (str): Path to the image file.
        model (nn.Module): The trained PyTorch model.
        topk (int): Number of top predictions to display.
    """
    # Predict the top classes and probabilities
    probs, classes = predict(image_path, model, topk)

    # Process the image
    processed_image = process_image(image_path)

    # Plot the image
    fig, (ax1, ax2) = plt.subplots(figsize=(6, 9), nrows=2)
    imshow(processed_image, ax=ax1, title="Input Image")

    # Plot the predictions
    ax2.barh(classes, probs, color='blue')  # Horizontal bar chart
    ax2.set_aspect(0.1)
    ax2.set_yticks(range(len(classes)))
    ax2.set_yticklabels(classes)
    ax2.set_xlim(0, 1)  # Limit probabilities to [0, 1]

    plt.tight_layout()
    plt.show()


## Reminder for Workspace users
If your network becomes very large when saved as a checkpoint, there might be issues with saving backups in your workspace. You should reduce the size of your hidden layers and train again.
    
We strongly encourage you to delete these large interim files and directories before navigating to another page or closing the browser tab.

In [ ]:
# TODO remove .pth files or move it to a temporary `~/opt` directory in this Workspace
!rm checkpoint.pth

In [ ]:
from google.colab import files

# Save the notebook as an HTML file
!jupyter nbconvert --to html 'Image Classifier Project (1).ipynb'

# Download the HTML file
files.download('Image Classifier Project (1).html')
